# SARAH Toolbox

## Collection of models related to the 750 GeV diphoton excess

See for more details [1602.05581](http://arxiv.org/abs/1602.05581)

[![Binder](http://mybinder.org/badge.svg)](http://mybinder.org/repo/restrepo/BSM-Toolbox)

[Jupyter home](./) (Files, New -> Terminal, etc)
[![Home](http://www.incredimail.com/images/nav%20bar/home-icon.png)](./)

### Highly recommended:
[Run from a terminal](../terminals/1):
[![T](http://mysitemyway.com/_shared/images/icons/full_color/terminal.png)](../terminals/1)


## Implemented models

* `SARAH/Models/SSDM/` (for test purposes)
* `SARAH/Models/diphotonModels/LRmodels/darkLR/`
* `SARAH/Models/diphotonModels/LRmodels/tripletLR/`  (with additional Z_2 and T_i dark matter)

## Test
Below we check the SSDM models. It is recomended that the commands be executed [from a terminal](../terminals/1).

Be sure that `SPHENO/Makefile` have `gfortan` uncommented:

In [1]:
cat  SPHENO/Makefile | grep "F90 ="

# F90 = nagfor
F90 = gfortran
# F90 = g95
# F90 = lf95
# F90 = ifort


In [1]:
%%bash
./compile_spheno_directly.sh SSDM

make: Entering directory '/home/restrepo/prog/toolbox/toolbox/SPHENO'
cd /home/restrepo/prog/toolbox/toolbox/SPHENO/SSDM ; make F90=gfortran version=330.00
make[1]: Entering directory '/home/restrepo/prog/toolbox/toolbox/SPHENO/SSDM'
cd ../src ; make F90=gfortran 
make[2]: Entering directory '/home/restrepo/prog/toolbox/toolbox/SPHENO/src'
gfortran -o SPheno -O   SPheno3.o ../lib/../lib/libSPheno.a
mv SPheno ../bin
make[2]: Leaving directory '/home/restrepo/prog/toolbox/toolbox/SPHENO/src'
make F90=gfortran ../lib/libSPhenoSSDM.a 
make[2]: Entering directory '/home/restrepo/prog/toolbox/toolbox/SPHENO/SSDM'
make[2]: Nothing to be done for '../lib/libSPhenoSSDM.a'.
make[2]: Leaving directory '/home/restrepo/prog/toolbox/toolbox/SPHENO/SSDM'
make F90=gfortran SPhenoSSDM.o 
make[2]: Entering directory '/home/restrepo/prog/toolbox/toolbox/SPHENO/SSDM'
gfortran -c -g -ffree-line-length-none -J../include -I../include   SPhenoSSDM.f90
make[2]: Leaving directory '/home/restrepo/prog/toolbox/to

make a work dir

In [2]:
mkdir work

In [3]:
cd work

/home/restrepo/prog/toolbox/toolbox/work


If neccesary, turn ON tachyonic spectrum with `52 1 #...`

In [6]:
cat ../SPHENO/SSDM/LesHouches.in.SSDM | grep 'Write spectrum in case of tachyonic states' 

 52 0               # Write spectrum in case of tachyonic states 


In [5]:
%%bash
../SPHENO/bin/SPhenoSSDM ../SPHENO/SSDM/LesHouches.in.SSDM

 Calculating branching ratios and decay widths
 Calculating low energy constraints
 Writing output files
 Finished!


In [7]:
%%bash
cd ../micromegas
make
make
cd SSDM
make main=CalcOmega.cpp

make -C CalcHEP_src
make[1]: Entering directory '/home/restrepo/prog/toolbox/toolbox/micromegas/CalcHEP_src'
./getFlags
C compiler detected
: shared liblary generation OK
X11 OK
Fortran compiler is detected
gfortran OK
FFLAGS OK
./sbin/setPath /home/restrepo/prog/toolbox/toolbox/micromegas/CalcHEP_src
chmod 755 mkWORKdir
make -C c_source
make[2]: Entering directory '/home/restrepo/prog/toolbox/toolbox/micromegas/CalcHEP_src/c_source'
make -C getmem 
make[3]: Entering directory '/home/restrepo/prog/toolbox/toolbox/micromegas/CalcHEP_src/c_source/getmem'
make[3]: Nothing to be done for 'all'.
make[3]: Leaving directory '/home/restrepo/prog/toolbox/toolbox/micromegas/CalcHEP_src/c_source/getmem'
make -C chep_crt
make[3]: Entering directory '/home/restrepo/prog/toolbox/toolbox/micromegas/CalcHEP_src/c_source/chep_crt'
make[3]: Nothing to be done for 'all'.
make[3]: Leaving directory '/home/restrepo/prog/toolbox/toolbox/micromegas/CalcHEP_src/c_source/chep_crt'
make -C service2
make[3]: Ent

Run micromegas

In [8]:
%%bash
../micromegas/SSDM/CalcOmega SPheno.spc.SSDM

PROCESS: ~Ss,~Ss->AllEven,1*x{,h,g,A,Z,Wp,Wm,nu1,Nu1,nu2,Nu2,nu3,Nu3,d1,D1,d2,D2,d3,D3,u1,U1,u2,U2,u3,U3,e1,E1,e2,E2,e3,E3

Masses of odd sector Particles:
~Ss  : MSs   =    28.4 || 
Xf=1.61e+01 Omega h^2=2.23e+01

# Channels which contribute to 1/(omega) more than 1%.
# Relative contributions in % are displayed
   85% ~Ss ~Ss ->d3 D3 
    8% ~Ss ~Ss ->e3 E3 
    4% ~Ss ~Ss ->u2 U2 
    2% ~Ss ~Ss ->g g 
